In [31]:
import multiversx_utils as mu
import json
import pandas as pd
import os

In [32]:
data_folder_path = 'data'
collections = ['CRMYTH-546419', 'CRWEAPONS-e5ab49', 'GSPACEAPE-08bc2b', 'CEA-2d29f9', 'CRCHAMPS-d0265d', 'CRHEROES-9edff2', 'EAPES-8f3c1f']
operations = {
    'info': True,
    'nfts_raw': True,
    'nfts_processed': True,
    'txs': False
}
params = {
    'CRMYTH-546419': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/']
    },
    'CRWEAPONS-e5ab49': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/']
    },
    'GSPACEAPE-08bc2b': {
        'sleep_time': 0.2,
        'whitelist': ['https://metadata.verko.io/dynamic/']
    },
    'CEA-2d29f9': {
        'sleep_time': 0.2,
        'whitelist': ['https://metadata.verko.io/dynamic/']
    },
    'CRCHAMPS-d0265d': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/', 'https://metadata.cantinaroyale.io/metadata/']
    },
    'CRHEROES-9edff2': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/', 'https://metadata.cantinaroyale.io/metadata/']
    },
    'EAPES-8f3c1f': {
        'sleep_time': 0,
        'whitelist': []
    }
}

In [33]:
def get_collection_info(collection_name, collection_folder_path):
    collection_info = mu.get_collection_info(collection_name)
    filename = os.path.join(collection_folder_path, 'info.json')
    with open(filename, 'w') as f:
        json.dump(collection_info, f, indent=4)
    return

def get_collection_nfts_raw(collection_name, collection_folder_path, params):
    collection_nfts = mu.get_collection_nfts(collection_name)
    sleep_time = params[collection_name]['sleep_time']
    whitelist = params[collection_name]['whitelist']
    collection_offchain_data = mu.get_collection_offchain_data(collection_nfts, sleep_time=sleep_time, whitelist=whitelist)
    for identifier in collection_nfts:
        collection_nfts[identifier]['offchain_data'] = collection_offchain_data[identifier]
    filename = os.path.join(collection_folder_path, 'nfts_raw.json')
    with open(filename, 'w') as f:
        json.dump(collection_nfts, f, indent=4)
    return

def get_collection_nfts_processed(collection_name, collection_folder_path):
    filename = os.path.join(collection_folder_path, 'nfts_raw.json')
    with open(filename, 'r') as f:
        collection_nfts = json.load(f)
    collection = {}
    for nft in collection_nfts.values():
        nft_data = mu.parse_nft_data(nft)
        for key, value in nft_data.items():
            if key not in collection:
                collection[key] = []
            collection[key].append(value)
    filename = os.path.join(collection_folder_path, 'nfts_processed.csv')
    collection = pd.DataFrame(collection)
    collection.to_csv(filename, index=False)
    collection.index = collection['identifier']
    collection = collection.to_dict('index')
    for identifier, data in collection.items():
        for key, value in data.items():
            if pd.isna(value):
                collection[identifier][key] = None
    filename = os.path.join(collection_folder_path, 'nfts_processed.json')
    with open(filename, 'w') as f:
        json.dump(collection, f, indent=4)
    return

def get_collection_txs(collection_name, collection_folder_path):
    collection_txs = mu.get_collection_txs(collection_name)
    filename = os.path.join(collection_folder_path, 'txs.json')
    with open(filename, 'w') as f:
        json.dump(collection_txs, f, indent=4)
    return

In [34]:
for collection_name in collections:
    collection_folder_path = os.path.join(data_folder_path, collection_name)
    if not os.path.exists(collection_folder_path):
        os.makedirs(collection_folder_path)
    if operations['info']:
        get_collection_info(collection_name, collection_folder_path)
    if operations['nfts_raw']:
        get_collection_nfts_raw(collection_name, collection_folder_path, params)
    if operations['nfts_processed']:
        get_collection_nfts_processed(collection_name, collection_folder_path)
    if operations['txs']:
        get_collection_txs(collection_name, collection_folder_path)

get_collection_offchain_data('CRMYTH-546419'): 100%|██████████| 584/584 [10:20<00:00,  1.06s/it]  
